In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Dataset

In [20]:
# load admission dataset
admissions = pd.read_csv('admissions_clean.csv')

#https://mit-lcp.github.io/mimic-schema-spy/index.html

In [21]:
admissions

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,Readmitted,adm_disch_diff,disch_death_diff,erreg_erout_diff
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,...,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,0,1.144444,NaN,0.137500
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,1,5.496528,NaN,NaN
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,WHITE,NaN,NaN,BRAIN MASS,0,1,1,6.768056,NaN,NaN
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,...,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,0,2.856944,NaN,NaN
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,...,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,0,3.534028,NaN,0.143056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,...,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1,0,2.742361,NaN,0.122917
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,...,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,0,0.548611,0.0,0.154861
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,...,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1,0,8.203472,NaN,NaN
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0,1,41.445833,NaN,0.144444


In [22]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ROW_ID                58976 non-null  int64  
 1   SUBJECT_ID            58976 non-null  int64  
 2   HADM_ID               58976 non-null  int64  
 3   ADMITTIME             58976 non-null  object 
 4   DISCHTIME             58976 non-null  object 
 5   DEATHTIME             5854 non-null   object 
 6   ADMISSION_TYPE        58976 non-null  object 
 7   ADMISSION_LOCATION    58976 non-null  object 
 8   DISCHARGE_LOCATION    58976 non-null  object 
 9   INSURANCE             58976 non-null  object 
 10  LANGUAGE              58976 non-null  object 
 11  RELIGION              58976 non-null  object 
 12  MARITAL_STATUS        58976 non-null  object 
 13  ETHNICITY             58976 non-null  object 
 14  EDREGTIME             30877 non-null  object 
 15  EDOUTTIME          

In [23]:
admissions.shape

(58976, 23)

In [25]:
admissions.isnull().sum()

ROW_ID                      0
SUBJECT_ID                  0
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               53122
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
LANGUAGE                    0
RELIGION                    0
MARITAL_STATUS              0
ETHNICITY                   0
EDREGTIME               28099
EDOUTTIME               28099
DIAGNOSIS                  25
HOSPITAL_EXPIRE_FLAG        0
HAS_CHARTEVENTS_DATA        0
Readmitted                  0
adm_disch_diff              0
disch_death_diff        53122
erreg_erout_diff        28099
dtype: int64

Load Diagnosis_ICD table

In [26]:
# reading gzip file
file = 'C:/Users/aneja/Documents/Docs/Winter_2024/HealthCare Analytics/Project/New folder/DIAGNOSES_ICD.csv.gz'
diagnoses_df = pd.read_csv(file, compression='gzip')
diagnoses_df

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [27]:
diagnoses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB


In [28]:
# Select the relevant columns from diagnoses_df
icd9_codes = diagnoses_df[['ROW_ID', 'ICD9_CODE']]

# Merge the ICD9_CODE column with the admissions table using ROW_ID as the key
adm_icd_data = admissions.merge(icd9_codes, on='ROW_ID', how='left')

# Drop the ROW_ID column after the merge
adm_icd_data.drop(columns='ROW_ID', inplace=True)


In [29]:
adm_icd_data

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,Readmitted,adm_disch_diff,disch_death_diff,erreg_erout_diff,ICD9_CODE
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NOT SPECIFIED,...,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,0,1.144444,NaN,0.137500,E9317
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NOT SPECIFIED,...,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,1,5.496528,NaN,NaN,V3000
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,NaN,NaN,BRAIN MASS,0,1,1,6.768056,NaN,NaN,V053
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NOT SPECIFIED,...,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,0,2.856944,NaN,NaN,V290
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NOT SPECIFIED,...,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,0,3.534028,NaN,0.143056,40391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,...,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1,0,2.742361,NaN,0.122917,2819
58972,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,0,0.548611,0.0,0.154861,4589
58973,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,...,NaN,NaN,RENAL CANCER/SDA,0,1,0,8.203472,NaN,NaN,7533
58974,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,...,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0,1,41.445833,NaN,0.144444,7140


In [30]:
adm_icd_data.shape

(58976, 23)

In [31]:
adm_icd_data.isnull().sum()

SUBJECT_ID                  0
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               53122
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
LANGUAGE                    0
RELIGION                    0
MARITAL_STATUS              0
ETHNICITY                   0
EDREGTIME               28099
EDOUTTIME               28099
DIAGNOSIS                  25
HOSPITAL_EXPIRE_FLAG        0
HAS_CHARTEVENTS_DATA        0
Readmitted                  0
adm_disch_diff              0
disch_death_diff        53122
erreg_erout_diff        28099
ICD9_CODE                   7
dtype: int64

In [51]:
#Handling missing Diagnosis codes
adm_icd_data['DIAGNOSIS'] = adm_icd_data['DIAGNOSIS'].fillna('NO DIAGNOSIS')
adm_icd_data


,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,Readmitted,adm_disch_diff,disch_death_diff,erreg_erout_diff,ICD9_CODE
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NOT SPECIFIED,...,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,0,1.144444,NaN,0.137500,E9317
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NOT SPECIFIED,...,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,1,5.496528,NaN,NaN,V3000
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,NaN,NaN,BRAIN MASS,0,1,1,6.768056,NaN,NaN,V053
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NOT SPECIFIED,...,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,0,2.856944,NaN,NaN,V290
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NOT SPECIFIED,...,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,0,3.534028,NaN,0.143056,40391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,...,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1,0,2.742361,NaN,0.122917,2819
58972,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,0,0.548611,0.0,0.154861,4589
58973,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,...,NaN,NaN,RENAL CANCER/SDA,0,1,0,8.203472,NaN,NaN,7533
58974,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,...,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0,1,41.445833,NaN,0.144444,7140


In [53]:
#Handling of missing ICD codes
adm_icd_data['ICD9_CODE'] = adm_icd_data['ICD9_CODE'].fillna('NO ICD Codes')
adm_icd_data

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,Readmitted,adm_disch_diff,disch_death_diff,erreg_erout_diff,ICD9_CODE
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NOT SPECIFIED,...,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,0,1.144444,NaN,0.137500,E9317
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NOT SPECIFIED,...,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,1,5.496528,NaN,NaN,V3000
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,NaN,NaN,BRAIN MASS,0,1,1,6.768056,NaN,NaN,V053
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NOT SPECIFIED,...,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,0,2.856944,NaN,NaN,V290
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NOT SPECIFIED,...,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,0,3.534028,NaN,0.143056,40391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,...,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1,0,2.742361,NaN,0.122917,2819
58972,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1,0,0.548611,0.0,0.154861,4589
58973,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,...,NaN,NaN,RENAL CANCER/SDA,0,1,0,8.203472,NaN,NaN,7533
58974,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,...,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0,1,41.445833,NaN,0.144444,7140


In [54]:
adm_icd_data.isnull().sum()

SUBJECT_ID                  0
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               53122
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
LANGUAGE                    0
RELIGION                    0
MARITAL_STATUS              0
ETHNICITY                   0
EDREGTIME               28099
EDOUTTIME               28099
DIAGNOSIS                   0
HOSPITAL_EXPIRE_FLAG        0
HAS_CHARTEVENTS_DATA        0
Readmitted                  0
adm_disch_diff              0
disch_death_diff        53122
erreg_erout_diff        28099
ICD9_CODE                   0
dtype: int64

In [55]:
adm_icd_data['ICD9_CODE'].value_counts()

4019     1939
4280     1390
41401    1244
42731    1213
V053      977
         ... 
75453       1
E9356       1
9581        1
6988        1
86404       1
Name: ICD9_CODE, Length: 3406, dtype: int64

In [57]:
# Classifying the diagnosis on the basis of ICD 9 codes
adm_icd_data['Diagnosis_class'] = ''

In [78]:
icd9_ranges = {
    (1, 139): 'Infectious and Parasitic Diseases',
    (140, 208): 'Neoplasms',
    (210, 229): 'Benign Neoplasms',
    (240, 279): 'Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders',
    (280, 289): 'Diseases of the Blood and Blood-forming Organs',
    (290, 319): 'Mental Disorders',
    (320, 389): 'Diseases of the Nervous System and Sense Organs',
    (390, 459): 'Diseases of the Circulatory System',
    (460, 519): 'Diseases of the Respiratory System',
    (520, 579): 'Diseases of the Digestive System',
    (580, 629): 'Diseases of the Genitourinary System',
    (630, 679): 'Complications of Pregnancy, Childbirth, and the Puerperium',
    (680, 709): 'Diseases of the Skin and Subcutaneous Tissue',
    (710, 739): 'Diseases of the Musculoskeletal System and Connective Tissue',
    (740, 759): 'Congenital Anomalies',
    (760, 779): 'Certain Conditions originating in the Perinatal Period',
    (780, 799): 'Symptoms, Signs and Ill-defined Conditions',
    (800, 999): 'Injury and Poisoning'}


In [103]:
# Define the function to classify ICD9 codes based on the provided criteria
def classify_icd9(icd9_code):
    if isinstance(icd9_code, int):
        # If the ICD9 code is an integer, extract the first 3 characters as prefix
        prefix = icd9_code[:3]
        if prefix.isdigit():
            # If the prefix is numeric, check if it falls within the specified ranges
            for icd_range, diag_class in icd9_ranges.items():
                if int(prefix) in range(icd_range[0], icd_range[1] + 1):
                    return diag_class
        elif prefix.startswith('E'):
            return 'Supplementary Classification of External Causes of Injury and Poisoning'
        elif prefix.startswith('V'):
            return 'Supplementary Classification of Factors influencing Health Status and Contact with Health Services'
        elif prefix.startswith('M'):
            return 'Morphology of Neoplasms'
        elif 'NO ICD Codes' in icd9_code:
            return 'NO ICD Codes'
        else:
            return 'Other'  # Default category for unmatched codes
    else:
        return 'Other'  # Default category for non-integer codes

# Apply the function to create the 'Diag_Class' column in adm_icd_data DataFrame
adm_icd_data['Diag_Class'] = adm_icd_data['ICD9_CODE'].apply(classify_icd9)


In [109]:
# Create a function to classify ICD 9 codes based on specified criteria
def classify_icd9(icd9_code):
    if isinstance(icd9_code, str):  # Check if the code is a string
        prefix = icd9_code[:3]  # Extract the first three characters
        for icd_range, diag_class in icd9_ranges.items():
            if prefix.isdigit():  # Check if the code is numeric
                if int(prefix) in range(icd_range[0], icd_range[1] + 1):
                    return diag_class
            else:
                if prefix in icd_range:  # Check if the prefix falls within the range
                    return diag_class
        if prefix.startswith('E'):
            return 'Supplementary Classification of External Causes of Injury and Poisoning'
        elif prefix.startswith('V'):
            return 'Supplementary Classification of Factors influencing Health Status and Contact with Health Services'
        elif prefix.startswith('M'):
            return 'Morphology of Neoplasms'
        elif 'NO ICD Codes' in icd9_code:
            return 'NO ICD Codes'
        else:
            return 'Other'  # Default category for unmatched codes
    elif isinstance(icd9_code, float):  # Check if the code is a float
        return 'Other'  # Default category for floating-point values
    return 'Other'  # Default category for other non-numeric or unmatched codes

# Apply the function to create the Diag_Class column
adm_icd_data['Diag_Class'] = adm_icd_data['ICD9_CODE'].apply(classify_icd9)

In [110]:
adm_icd_data

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,Readmitted,adm_disch_diff,disch_death_diff,erreg_erout_diff,ICD9_CODE,Diagnosis_class,Diag_Class
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NOT SPECIFIED,...,BENZODIAZEPINE OVERDOSE,0,1,0,1.144444,NaN,0.137500,E9317,,Supplementary Classification of External Cause...
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NOT SPECIFIED,...,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,1,5.496528,NaN,NaN,V3000,,Supplementary Classification of Factors influe...
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,BRAIN MASS,0,1,1,6.768056,NaN,NaN,V053,,Supplementary Classification of Factors influe...
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NOT SPECIFIED,...,INTERIOR MYOCARDIAL INFARCTION,0,1,0,2.856944,NaN,NaN,V290,,Supplementary Classification of Factors influe...
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NOT SPECIFIED,...,ACUTE CORONARY SYNDROME,0,1,0,3.534028,NaN,0.143056,40391,,Diseases of the Circulatory System
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,...,TRAUMA,0,1,0,2.742361,NaN,0.122917,2819,,Diseases of the Blood and Blood-forming Organs
58972,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,SAH,1,1,0,0.548611,0.0,0.154861,4589,,Diseases of the Circulatory System
58973,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,...,RENAL CANCER/SDA,0,1,0,8.203472,NaN,NaN,7533,,Congenital Anomalies
58974,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,...,S/P FALL,0,0,1,41.445833,NaN,0.144444,7140,,Diseases of the Musculoskeletal System and Con...


In [111]:
# Calculate the value counts of each class under Diag_Class
class_counts = adm_icd_data['Diag_Class'].value_counts()

# Calculate the percentage of occurrence for each class
class_percentages = (class_counts / len(adm_icd_data)) * 100

# Display the percentages
class_percentages


Diseases of the Circulatory System                                                                    21.898738
Supplementary Classification of Factors influencing Health Status and Contact with Health Services    12.569520
Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders                                  9.790423
Diseases of the Respiratory System                                                                     7.302971
Injury and Poisoning                                                                                   7.104585
Certain Conditions originating in the Perinatal Period                                                 5.807447
Diseases of the Digestive System                                                                       5.156335
Diseases of the Genitourinary System                                                                   4.740912
Diseases of the Blood and Blood-forming Organs                                                         3

In [112]:
# Calculate the value counts of each class under Diag_Class
code_counts = adm_icd_data['ICD9_CODE'].value_counts()

# Calculate the percentage of occurrence for each class
code_percentages = (class_counts / len(adm_icd_data)) * 100

# Display the percentages
code_percentages


Diseases of the Circulatory System                                                                    21.898738
Supplementary Classification of Factors influencing Health Status and Contact with Health Services    12.569520
Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders                                  9.790423
Diseases of the Respiratory System                                                                     7.302971
Injury and Poisoning                                                                                   7.104585
Certain Conditions originating in the Perinatal Period                                                 5.807447
Diseases of the Digestive System                                                                       5.156335
Diseases of the Genitourinary System                                                                   4.740912
Diseases of the Blood and Blood-forming Organs                                                         3

In [113]:
data = adm_icd_data[adm_icd_data['Diag_Class'] == 'Diseases of the Circulatory System']
data

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,Readmitted,adm_disch_diff,disch_death_diff,erreg_erout_diff,ICD9_CODE,Diagnosis_class,Diag_Class
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NOT SPECIFIED,...,ACUTE CORONARY SYNDROME,0,1,0,3.534028,NaN,0.143056,40391,,Diseases of the Circulatory System
5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NOT SPECIFIED,...,V-TACH,0,1,0,6.988889,NaN,NaN,4440,,Diseases of the Circulatory System
21,41,101757,2132-12-31 10:30:00,2133-01-27 15:45:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DISC-TRAN TO FEDERAL HC,Private,NOT SPECIFIED,...,RIGHT BRAIN STEM LESION/SDA,0,1,0,27.218750,NaN,NaN,431,,Diseases of the Circulatory System
46,43,146828,2186-10-01 23:15:00,2186-10-11 18:00:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,...,T12 FRACTURE,0,1,0,9.781250,NaN,0.169444,4280,,Diseases of the Circulatory System
49,46,144073,2133-02-20 18:21:00,2133-02-24 14:42:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,NOT SPECIFIED,...,GASTROINTESTINAL BLEED,0,1,0,3.847917,NaN,NaN,4019,,Diseases of the Circulatory System
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58943,98724,186754,2143-08-22 16:01:00,2143-09-02 17:15:00,NaN,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,Medicare,ENGL,...,CELLULITIS,0,1,0,11.051389,NaN,0.249306,3970,,Diseases of the Circulatory System
58944,98733,152232,2102-10-08 12:32:00,2102-10-16 15:30:00,NaN,URGENT/EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,ENGL,...,CORONARY ARTERY DISEASE,0,1,0,8.123611,NaN,NaN,4280,,Diseases of the Circulatory System
58945,98734,123336,2146-10-02 21:31:00,2146-10-06 16:00:00,NaN,URGENT/EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,ENGL,...,UTI;PYELONEPHRITIS,0,1,0,3.770139,NaN,0.355556,4019,,Diseases of the Circulatory System
58972,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,URGENT/EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,SAH,1,1,0,0.548611,0.0,0.154861,4589,,Diseases of the Circulatory System
